In [15]:
# current working dir is /var/lib/jupyter/notebooks
import numpy as np
import opentrons.execute
from opentrons import protocol_api
from src.pipette_viscous import transfer_viscous,aspirate_viscous, dispense_viscous
import src.scheduler as scheduler
from src.evap_standard_curve import *

# start the protocol context
protocol = opentrons.execute.get_protocol_api("2.11")

# home is required
protocol.set_rail_lights(False)
protocol.home()

In [2]:
incubator = protocol.load_module("temperature module", 11)

rack = incubator.load_labware("opentrons_24_aluminumblock_fisher_1.5ml_centrifuge")    # the rack is mounted upon the temp. module (don't specify slot)

tip_rack_1 = protocol.load_labware("geb_taller_96_tiprack_10ul", '8')    # will consume 380 tips in total, about 4 boxes

plate = protocol.load_labware("biorad_96_wellplate_200ul_pcr", '3')

pipette = protocol.load_instrument("p10_single", "left", tip_racks = [tip_rack_1])

In [11]:
RXN_TUBE_1 = rack.wells_by_name()['A1']
RXN_TUBE_2 = rack.wells_by_name()['B1']
RXN_TUBE_3 = rack.wells_by_name()['C1']
# DIS_TUBE = rack.wells_by_name()['C1']
FROM_TUBE = rack.wells_by_name()['A3']

In [4]:
v_s = 9

def calibrated(v):
    return v + 0

def sample_test():
    "take a sample from the tube to a 96-well plate"

    # pipette the dye
    #transfer_viscous(pipette, protocol, v_s / 3, DYE_TUBE, cur_well)

    # pipette the sample
    aspirate_viscous(pipette, protocol, calibrated(v_s), RXN_TUBE, asp_height=2)
    dispense_viscous(pipette, protocol, calibrated(v_s), DIS_TUBE, if_mix=True)
    

v_g = 9

def grow_test(v_g):
    "dilute the reaction with U-KaiC, KaiA, and KaiB by volume v_g"
    
    v_g_per_tube = np.round(v_g / 3, 1)
    
    for i in range(3):
        # it's also okay to mix KaiA and KaiB in one tube, then all conc are 2x
        # do not dispense at the bottom. Defaults are 1mm from the bottom
        transfer_viscous(pipette, protocol, calibrated(v_g_per_tube), FROM_TUBE, RXN_TUBE, if_mix=True) 
    

In [6]:
protocol.set_rail_lights(True)

for i in range(3):
    grow_test(v_g)
    sample_test()

protocol.set_rail_lights(False)

In [14]:
vol = 1

protocol.set_rail_lights(True)

for i in range(5):
    aspirate_viscous(pipette, protocol, vol, FROM_TUBE, asp_height=2)
    dispense_viscous(pipette, protocol, vol, RXN_TUBE_1, if_mix=True)
    
for i in range(5):
    aspirate_viscous(pipette, protocol, vol, FROM_TUBE, asp_height=2)
    dispense_viscous(pipette, protocol, vol, RXN_TUBE_2, if_mix=True)
    
for i in range(5):
    aspirate_viscous(pipette, protocol, vol, FROM_TUBE, asp_height=2)
    dispense_viscous(pipette, protocol, vol, RXN_TUBE_3, if_mix=True)
    
protocol.set_rail_lights(False)